## Near-field Ptychography Pipeline

In [1]:
import numpy
import matplotlib.pyplot as plt

import sscCdi

print(f'sscCdi version = {sscCdi.__version__}')

sscCdi version = 0.7.14


In [ ]:
input_dict = 
{'path': '/ibira/lnls/labs/tepui/home/yuri.tonin/00000000/data/ptycho3d/complex_phantom/',
 'n_cpus': 32,
 'energy': 22, # keV
 'distance': 0.995721938, # z2 detector-sample distance
 'source_distance': 0.004278062, # z1 source-sample distance. should be 0 for paralel beam case
 'detector_pixel_size': 1.44e-06, # meters
 'object_padding': 100, # pixels 
 'probe_support': None,
 'fresnel_regime' = True,
 'incoherent_modes': 1,


'algorithms': {
    
'1': {'name': 'ePIE_python',
        'iterations': 100,
        'regularization_object': 0.25,
        'regularization_probe': 0.5,
        'step_object': 0.5,
        'step_probe': 1,
        'use_mPIE': False, # friction and momentum counter only relevant if this is True
        'mPIE_friction_obj': 0.9,
        'mPIE_friction_probe': 0.99,
        'mPIE_momentum_counter': 10,
        },
    
'2':  {'name': 'RAAR_python',
        'iterations': 30,
        'regularization_object': 0.25,
        'regularization_probe': 0.5,
        'step_object': 0.9,
        'step_probe': 0.9,
        }
}

## Load data

In [ ]:
DPs = np.load(input_dict['path'])
print(f'Data shape = {DPs.shape}')
print("Size of the array:", DPs.nbytes / (1024**3), "GB") # Get the size of the array in bytes and convert the size to gigabytes

### Load positions

In [ ]:
scan_positions = sscCdi.read_mogno_probe_positions(input_dict,DPs.shape)
print(f'Positions shape = {scan_positions.shape}')

### Set initial object

In [ ]:
inputs = sscCdi.set_object_shape(inputs, DPs.shape, positions)
obj_guess = np.ones(inputs["object_shape"])
print(f'Object shape = {inputs["object_shape"]}')

### Initial probe: use flat if available, otherwise, constant array of 1's

In [ ]:
probe_guess = np.load('empty.npy') + 1j
print(f'Probe shape = {probe.shape}')

### Call Ptychography

In [ ]:
obj, probes, input_dict = sscCdi.call_ptychography(input_dict,DPs,scan_positions,initial_obj=obj_guess,initial_probe=probe_guess)
print(f'Object shape = {obj.shape} \t Probe shape = {probes.shape}')

In [ ]:
sscCdi.deploy_visualizer(obj,axis=0,cmap='cividis')

In [ ]:
fig, ax = plt.subplots(2,4,figsize=(13,8),dpi=100)
ax[0,0].imshow(np.abs(RAAR_obj)), ax[0,0].set_title('Obj Mag')
ax[0,0].plot(positions[:,1],positions[:,0],'.',color='r')
ax[0,1].imshow(np.angle(RAAR_obj)), ax[0,1].set_title('Obj Phase')
ax[0,2].imshow(np.abs(empty),vmax=6000), ax[0,2].set_title('Flat')
ax[1,0].imshow(np.abs(RAAR_probes[0])), ax[1,0].set_title('Probe Mag')
ax[1,1].imshow(np.angle(RAAR_probes[0])), ax[1,1].set_title('Probe Phase')
ax[1,2].imshow(np.abs(propagated_probe),vmax=0.5), ax[1,2].set_title('Mag Propagated')
ax[1,3].imshow(np.angle(propagated_probe),vmax=0.1,vmin=-0.1), ax[1,3].set_title('Phase Propagated')
ax[0,3].plot(RAAR_error,'o-',label='Error')
ax[0,3].set_xlabel('Iterations')
ax[0,3].legend(), ax[0,3].grid()

fig.suptitle('Tim Salditt RHD sample - Near Field Ptychography')

for i, ax in enumerate(ax.ravel()):
    if i==3 or i==0 or i ==4: continue
    ax.set_xticks([])
    ax.set_yticks([])